# Reading tables and storing information as tables and databases

In [2]:
# Configure plots for inline use in Jupyter Notebook
%matplotlib inline

import datetime as dt

# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import rasterio
# Various utilities
import json
import xarray as xr
import shapely.geometry
import IPython.display
import zarr

# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SHConfig,
    bbox_to_dimensions,
    os_utils,
)

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")

print ("START the work ......")

START the work ......


In [3]:
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
from pathlib import Path  

### reading raster
raster_2band = "./../../../s3/interim/LU001C/LU001C.tiff"
print (raster_2band)

with rio.Env():
    with rio.open(raster_2band) as src:
        crs = src.crs

        # create 1D coordinate arrays (coordinates of the pixel center)
        xmin, ymax = np.around(src.xy(0.00, 0.00), 9)  # src.xy(0, 0)
        xmax, ymin = np.around(src.xy(src.height-1, src.width-1), 9)  # src.xy(src.width-1, src.height-1)
        x = np.linspace(xmin, xmax, src.width)
        y = np.linspace(ymax, ymin, src.height)  # max -> min so coords are top -> bottom


        # create 2D arrays
        xs, ys = np.meshgrid(x, y)
        b1 = src.read(1)
        b2 = src.read(2)

        # Apply NoData mask
        mask = src.read_masks(1) > 0
        xs, ys, b1,b2 = xs[mask], ys[mask], b1[mask],b2[mask]

data = {"X": pd.Series(xs.ravel()),
        "Y": pd.Series(ys.ravel()),
        "city_code": pd.Series(b1.ravel()),
        "urban_atlas_2018": pd.Series(b2.ravel())
       
       }

df = pd.DataFrame(data=data)
geometry = gpd.points_from_xy(df.X, df.Y)
gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

print(gdf.head())

print ("------------")

### combine different dimesnions: 
combine_table_cube_urban=df.groupby(['city_code','urban_atlas_2018']).size().reset_index().rename(columns={0:'count'})


### writting data to table
combine_table_cube_urban.to_csv(index=False)
ouptut_table = "./../../../s3/data/c001_city_cube/tables/urban_cube_v2.csv"
filepath = Path(ouptut_table)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
combine_table_cube_urban.to_csv(filepath)  



./../../../s3/interim/LU001C/LU001C.tiff
          X          Y  city_code  urban_atlas_2018                  geometry
0  6.069266  49.654853      65535             31000  POINT (6.06927 49.65485)
1  6.069399  49.654853      65535             31000  POINT (6.06940 49.65485)
2  6.069532  49.654853      65535             31000  POINT (6.06953 49.65485)
3  6.069665  49.654853      65535             31000  POINT (6.06967 49.65485)
4  6.069798  49.654853      65535             31000  POINT (6.06980 49.65485)
------------


NameError: name 'test' is not defined